In [ ]:
# Mount & imports
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix
)
import warnings
warnings.filterwarnings("ignore")

# Load, sort, and compute lagged return
df = pd.read_csv('/content/drive/MyDrive/MRP/final_dataset.csv', parse_dates=['date'])
df = df.sort_values(['symbol','date']).reset_index(drop=True)

# compute 1-day lagged return and drop any rows where it’s NaN
df['return_1d_lag1'] = df.groupby('symbol')['return_1d'].shift(1)
df = df.dropna(subset=['return_1d_lag1']).reset_index(drop=True)

# Chronological splits
train = df[   df['date'] <= '2021-12-31']
val   = df[(df['date'] > '2021-12-31') & (df['date'] <= '2022-12-31')]
test  = df[   df['date'] >  '2022-12-31']

# Per-symbol ARIMA on the lagged series
all_preds = []
for sym, _ in df.groupby('symbol'):
    # extract each symbol’s train-lagged series and test indices
    tr_idx = train[train['symbol']==sym].index
    te_idx =  test[test['symbol']==sym].index
    if len(tr_idx) < 30 or len(te_idx)==0:
        continue

    y_train = train.loc[tr_idx, 'return_1d_lag1']
    y_true  = test.loc[te_idx,  'target']

    try:
        # fit ARIMA(1,0,1) on the lagged returns
        m = ARIMA(y_train, order=(1,0,1)).fit()

        # forecast “lagged return” for each test index
        fc = m.forecast(steps=len(te_idx))
        fc.index = te_idx

        # assemble a small DataFrame for this symbol
        dfp = pd.DataFrame({
            'symbol'      : sym,
            'date'        : test.loc[te_idx, 'date'],
            'pred_return' : fc,
            'true_target' : y_true
        }, index=te_idx)
        dfp['pred_target'] = (dfp['pred_return'] > 0).astype(int)
        all_preds.append(dfp)

    except Exception:
        # in case ARIMA fails to converge
        continue

# Aggregate & Binarize
pred_df = pd.concat(all_preds).sort_index()

# Evaluate on Test Set
y_true = pred_df['true_target']
y_pred = pred_df['pred_target']
y_prob = pred_df['pred_return']

print("ARIMA(on return_1d_lag1) Performance on Test Set:")
print(f"  Accuracy   : {accuracy_score(y_true, y_pred):.4f}")
print(f"  Precision  : {precision_score(y_true, y_pred):.4f}")
print(f"  Recall     : {recall_score(y_true, y_pred):.4f}")
print(f"  F1 Score   : {f1_score(y_true, y_pred):.4f}")
print(f"  ROC AUC    : {roc_auc_score(y_true, y_prob):.4f}")
print("  Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

ARIMA(on return_1d_lag1) Performance on Test Set:
  Accuracy   : 0.5073
  Precision  : 0.5092
  Recall     : 0.6150
  F1 Score   : 0.5571
  ROC AUC    : 0.5089
  Confusion Matrix:
[[111627 168846]
 [109659 175163]]
